## Formalne metode u softverskom inzenjerstvu- Anja Ljiljak1124/22

### Biblioteke:

Za rad sa neuronskim mrežama u ovom projektu koristim TensorFlow biblioteku.

Ovu biblioteku nismo koristili na času, pa stavljam instrukcije za instalaciju:

#### Instalacija TensorFlow biblioteke

Pokrenite sledeću komandu u vašem terminalu:

```bash
pip install tensorflow 

#### Učitavanje biblioteka 

In [12]:
import numpy as np        

# Biblioteka za rad sa neuronskim mrezama                               
import tensorflow as tf                                 
from sklearn.datasets import fetch_rcv1         
from sklearn.model_selection import train_test_split 

#### Učitavanje i priprema podataka

In [13]:
# Ucitavanje RCV1 skupa podataka
rcv1 = fetch_rcv1()   
X = rcv1.data
y = rcv1.target


# Podjela podataka na trening i test (50% i za trening i za test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Konvertuje sparse matrice u SparseTensor objekte potrebne za TensorFlow
X_train_sparse = tf.sparse.SparseTensor(indices=np.column_stack(X_train.nonzero()), values=X_train.data, dense_shape=X_train.shape)
X_test_sparse = tf.sparse.SparseTensor(indices=np.column_stack(X_test.nonzero()), values=X_test.data, dense_shape=X_test.shape)

# Konverzija labela(izlaz) u tenzor
y_train_tensor = tf.convert_to_tensor(y_train.toarray(), dtype=tf.float32)       
y_test_tensor = tf.convert_to_tensor(y_test.toarray(), dtype=tf.float32)

#### Definisanje arhitekture Neuronske mreže

In [14]:
#Dobijam upozorenja iako sam instalirala biblioteku, ali radi normalno
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

model = Sequential([
    Input(shape=(X_train_sparse.shape[1],)),
    #dodaje gusti sloj sa 512 neurona
    Dense(512, activation='relu'),
    Dropout(0.5),
    #zadnji gusti sloj(103 izlaznih  neurona)
    Dense(y_train_tensor.shape[1], activation='sigmoid')

])

#sprema se za testiranje i treniranje
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['precision', 'recall'])

#### Testiranje i treniranje Neuronske mreže

In [15]:
class MetricsCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        #evaluacija modela na testnim podacima na kraju svake epohe
        test_metrics = model.evaluate(X_test_sparse, y_test_tensor, verbose=0)
        print(f"Epoha {epoch+1}/{self.params['epochs']}, trening_preciznost: {logs['precision']:.4f}, trening_odziv: {logs['recall']:.4f}, test_preciznost: {logs['val_precision']:.4f}, test_odziv: {logs['val_recall']:.4f}")

checkpoint = ModelCheckpoint('best_model.keras', monitor='val_precision', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_precision', patience=5, mode='max')

# Treniranje i testiranje modela
history = model.fit(X_train_sparse, y_train_tensor, epochs=10, batch_size=128, validation_data=(X_test_sparse, y_test_tensor), callbacks=[MetricsCallback(), checkpoint, early_stopping])

Epoch 1/10
3143/3143 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - loss: 0.0826 - precision: 0.8037 - recall: 0.5009Epoha 1/10, trening_preciznost: 0.8917, trening_odziv: 0.6698, test_preciznost: 0.9182, test_odziv: 0.7943
3143/3143 ━━━━━━━━━━━━━━━━━━━━ 668s 212ms/step - loss: 0.0826 - precision: 0.8037 - recall: 0.5010 - val_loss: 0.0239 - val_precision: 0.9182 - val_recall: 0.7943
Epoch 2/10
3143/3143 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 0.0231 - precision: 0.9104 - recall: 0.8157Epoha 2/10, trening_preciznost: 0.9089, trening_odziv: 0.8215, test_preciznost: 0.9117, test_odziv: 0.8220
3143/3143 ━━━━━━━━━━━━━━━━━━━━ 714s 227ms/step - loss: 0.0231 - precision: 0.9104 - recall: 0.8157 - val_loss: 0.0221 - val_precision: 0.9117 - val_recall: 0.8220
Epoch 3/10
3143/3143 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step - loss: 0.0194 - precision: 0.9188 - recall: 0.8492Epoha 3/10, trening_preciznost: 0.9162, trening_odziv: 0.8493, test_preciznost: 0.9097, test_odziv: 0.8293
3143/3143 ━━━━━━━━━━━━━━━━━━━━ 7

#### Importovanje potrebnih biblioteka

In [1]:
import lightgbm as lgb
import optuna
import numpy as np
from sklearn.datasets import fetch_rcv1
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score
from scipy.sparse import csr_matrix


#### Ucitavanje i priprema RCV1 dataset-a

In [2]:
# Ucitavanje RCV1 dataset-a
rcv1 = fetch_rcv1(subset='train')
X = rcv1.data
y = rcv1.target

# Pretvaranje sparse matrice u gustu
y = y.toarray()

# Koristim 50% uzoraka
X, _, y, _ = train_test_split(X, y, train_size=0.5, stratify=np.argmax(y, axis=1))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=np.argmax(y, axis=1))


#### Standardizacija podataka kako bi model mogao efikasnije da trenira

In [3]:
#StandardScaler bez centriranja zbog sparse matrice
scaler = StandardScaler(with_mean=False)
#Prilagodjenje scalera na trening i testne podatke i primjena standardizacije
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### Kreiranje LightGBM dataset-a za trening i validaciju

In [4]:
train_data = lgb.Dataset(X_train, label=np.argmax(y_train, axis=1))
valid_data = lgb.Dataset(X_test, label=np.argmax(y_test, axis=1), reference=train_data)

#### Definisanje objective  funkcije koja ce se koristiti za optimizaciju hiperparametara

In [5]:
def objective(trial):
    param = {
        'objective': 'multiclass',
        'num_class': y.shape[1],  # RCV1 ima 103 klase
        'metric': 'multi_logloss',
        #iskljucivanje logovanja, minimizuje se kolicina prikazanih informacija tokom treninga
        'verbosity': -1,
        'boosting_type': 'gbdt', #Gradient Boosting Decision Trees
        #Regularizacija L1 i L2 smanjuju kompleksnost modela da bi se izbjegao overfitting
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        #broj listova 
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'feature_pre_filter': False  # Iskljucujemo pre-filtering
    }
    #Treniranje modela sa trenutnim parametrima i pracenje validacionih rezultata 
    model = lgb.train(param, train_data, valid_sets=[valid_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=10), lgb.log_evaluation(0)])
    #Pravljenje predikcija na testnom skupu koristeci najbolje iteracije modela, vraca vjerovatnoce za svaku klasu za svaki uzorak
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    if y_pred.ndim > 1:
        #Vraca indeks klase sa najvecom vjerovatnocom za svaki uzorak
        y_pred = np.argmax(y_pred, axis=1)
        #Racunanje preciznosti modela koristeci pravu klasu i predikcije
    precision = precision_score(np.argmax(y_test, axis=1), y_pred, average='weighted')
    return precision


#### Optimizacija hiperparametara koriscenjem Optuna biblioteke

In [6]:
study = optuna.create_study(direction='maximize')  # Maksimizacija preciznosti
study.optimize(objective, n_trials=50)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2024-06-30 12:01:07,460] A new study created in memory with name: no-name-e7789d00-8e32-4a8c-9ec2-559aba6737ac


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[74]	valid_0's multi_logloss: 0.757088


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-06-30 12:02:17,149] Trial 0 finished with value: 0.7806287675090874 and parameters: {'lambda_l1': 1.8058825326333135e-08, 'lambda_l2': 0.011964338539988762, 'num_leaves': 220, 'feature_fraction': 0.9756338269197193, 'bagging_fraction': 0.4384668251439875, 'bagging_freq': 4, 'min_child_samples': 55, 'learning_rate': 0.0675543190548982}. Best is trial 0 with value: 0.7806287675090874.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[69]	valid_0's multi_logloss: 0.761773


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-06-30 12:04:16,745] Trial 1 finished with value: 0.7912306603993167 and parameters: {'lambda_l1': 0.003597311493596239, 'lambda_l2': 0.14276755814816638, 'num_leaves': 166, 'feature_fraction': 0.5762855118240453, 'bagging_fraction': 0.7077349604660885, 'bagging_freq': 9, 'min_child_samples': 24, 'learning_rate': 0.08165312579161246}. Best is trial 1 with value: 0.7912306603993167.


...

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.70176


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-06-30 13:31:54,795] Trial 48 finished with value: 0.7904104516619228 and parameters: {'lambda_l1': 1.3363102004028158, 'lambda_l2': 3.166710667546944, 'num_leaves': 136, 'feature_fraction': 0.512491124729581, 'bagging_fraction': 0.7728986852574921, 'bagging_freq': 3, 'min_child_samples': 20, 'learning_rate': 0.07850742792067096}. Best is trial 44 with value: 0.7982770235060412.


Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's multi_logloss: 0.812704


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
[I 2024-06-30 13:34:33,767] Trial 49 finished with value: 0.7853725968307694 and parameters: {'lambda_l1': 0.02079819165683564, 'lambda_l2': 0.013135628464685007, 'num_leaves': 162, 'feature_fraction': 0.6048069088687937, 'bagging_fraction': 0.7958275027018878, 'bagging_freq': 6, 'min_child_samples': 11, 'learning_rate': 0.06398537823364654}. Best is trial 44 with value: 0.7982770235060412.


Number of finished trials: 50
Best trial:
  Value: 0.7982770235060412
  Params: 
    lambda_l1: 0.17522744413925673
    lambda_l2: 0.08870705429122101
    num_leaves: 130
    feature_fraction: 0.5454918173863869
    bagging_fraction: 0.7885201159827951
    bagging_freq: 10
    min_child_samples: 22
    learning_rate: 0.07120189868694556


#### Neuronska mreza pokazuje bolje performanse u smislu preciznosti
lambda_l1 i lambda_l2 su parametri koji kontrolisu regularizaciju tj. smanjuju kompleksnost modela da bi se izbjegao overfitting i njihov opseg je od 1x10^-8 do 10.0(logaritamska skala). Odabrala sam ovaj siroki opseg da bi omogucila modelu da pronadje balans izmedju overfittinga i underfittinga.

num_leaves je u opsegu od 2 do 256, jer bi manji broj listova doveo do jednostavnijeg modela i moglo bi doci do smanjenja overfittinga, dok veci broj listova omogucava modelu da nauci slozenije obrasce ali moze povecati rizik od overfittinga, dok za optimalan opseg Optuna moze testirati razlicite vrijednosti kako bi nasla najbolji balans izmedju slozenosti modela i performansi.

feature_fraction ima opseg od 0.4 do 1.0, siri opseg omogucava modelu da istrazi kako razlicite vrijednosti uticu na performanse, sto moze dovesti do boljih rezultata

bagging_fraction je u opsegu od 0.4 do 1.0 i pomaze u smanjeu  prekomjernog prilagodjenja, tako sto nasumicno bira podskup podataka za svako stablo, te  siri opseg omogucava bolju optimizaciju

bagging_freq odredjuje koliko cesto se podaci ponovo uzorkuju, ima opseg od 1 do 10 smatram da je to optimalan opseg, jer omogucava dovoljno frekventnu upotrebu bagginga, da se iskoriste njegove prednosti, a da se ne preoptereti proces treniranja

min_child_samples ima opseg od 5 do 100 jer ovaj opseg omogucava razlicite nivoe dijeljenja cvorova, manje vrijednosti mogu kreirati veoma male cvorove i to moze dovesti do overfittinga

learning_rate ima opseg od 0.001 do 0.1, niza stopa ucenja moze dovesti do boljih performansi i cesto omogucava modelu da pronadje bolji globalni minimum, dok siri opseg omogucava modelu da brze uci, ali moze preskociti globalni minimum

feature_pre_filter ima vrijednost False jer to omogucava da se koriste sve dostupne karakteristike, sto moze poboljsati performanse modela